In [3]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [26]:
url= f'https://scholar.google.com/scholar?cites=6625945508587280573&as_sdt=5,47&sciodt=0,47&hl=en' # user manual input
citation_key =f'gupta1975investors' # user manual input

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
page_contents=requests.get(url,headers=headers)
soup=BeautifulSoup(page_contents.text,'html.parser')

citation_dict={}
def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list

dsoup = copy.deepcopy(soup)
dsoup= cited_by_gscholar(dsoup)
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=dsoup
df = pd.DataFrame.from_dict(citation_dict)
df.to_csv('academic_citation_network.csv', mode='a')